In [4]:
#Import libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics, svm, naive_bayes
from PIL import Image

In [2]:
#Function to extract labels from files
def extract_labels(file_path):
    with open(file_path) as file_labels:

        file_lines = file_labels.readlines()

        file_shape_labels= [None] * len(file_lines)
        file_position_labels = [None] * len(file_lines)
        file_phoneme_labels = [None] * len(file_lines)
        file_second_phoneme_label = [None] * len(file_lines)
        
        for i, line in enumerate(file_lines):
            file_line = line.split()

            file_shape_labels[i]  = int(file_line[1])
            file_position_labels[i] = int(file_line[2])
            file_phoneme_labels[i] = int(file_line[3])
            file_second_phoneme_label[i] = int(file_line[4])
            
    return np.array(file_shape_labels), np.array(file_position_labels), np.array(file_phoneme_labels), np.array(file_second_phoneme_label)

In [3]:
#Funtion to randomly shuffle data
def shuffle_data(images, shape_labels, position_labels, phoneme_labels, second_phoneme_labels):
    #index = np.arange(len(labels))
    #np.shuffle(index)
    index = np.random.choice(np.arange(len(phoneme_labels)), replace = False, size = len(phoneme_labels))
    
    shuffled_images = images[index]
    shuffled_shape_labels = shape_labels[index]
    shuffled_position_labels = position_labels[index]
    shuffled_phoneme_labels = phoneme_labels[index]
    shuffled_second_phoneme_labels = second_phoneme_labels[index]
    return shuffled_images, shuffled_shape_labels, shuffled_position_labels, shuffled_phoneme_labels, shuffled_second_phoneme_labels

In [4]:
#Function to split data
def split_data(data, split_ratio=0.9):
    partition = round(len(data) * split_ratio)
    
    train = data[:partition]
    test = data[partition:]
    return train, test

In [6]:
#Function to prepare dataset as grey images to input network
def create_grey_dataset(image_dir):
    grey_dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        r, g, b = image_as_array[:,:,0], image_as_array[:,:,1], image_as_array[:,:,2]
        grey_image = (0.3*r) + (0.59*g) +(0.11*b)
        grey_dataset.append(grey_image)
    return np.array(grey_dataset)

In [7]:
#Function to concatenate outputs of two networks into single vector
def join_outputs(out1, out2):
    joint_output = []
    for i in range(len(out1)):
        joint = np.concatenate((out1[i],out2[i]))
        joint_output.append(joint)
    return np.array(joint_output)

In [1]:
#Function to plot training and validation loss and accuracy
#Obtained from https://www.tensorflow.org/tutorials/images/classification
def plot_training_results(history, epochs = 5):
    
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.show()
    return

In [8]:
#Open image folders
consonants_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/ConsonantImages/')
vowels_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/VowelImages/')
combined_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/PhonemeImages/')

#Open label files
consonant_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Consonant.txt"
vowel_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Vowel.txt"
combined_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_All.txt"

In [9]:
#Extract labels
(vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used) = extract_labels(vowel_labels)
(consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this) = extract_labels(consonant_labels)
(combined_shape_labels, combined_position_labels, combined_consonant_labels, combined_vowel_labels) = extract_labels(combined_labels)

In [32]:
#Grey Vowels Model

In [72]:
#Convert images of consonants to greyscale and prepare to input into network
grey_vowel_images = create_grey_dataset(vowels_dir)
grey_vowel_images = grey_vowel_images.reshape(len(grey_vowel_images), 256, 256, 1)

In [73]:
#Randomly shuffle consonant images and labels
grey_shuffled_vowels, grey_shuffled_vowel_shape_labels, grey_shuffled_vowel_position_labels, grey_shuffled_vowel_phoneme_labels, grey_non_used = shuffle_data(grey_vowel_images, vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used)

#Split training and test data
grey_vowel_train, grey_vowel_test = split_data(grey_shuffled_vowels)
grey_vowel_shape_labels_train, grey_vowel_shape_labels_test = split_data(grey_shuffled_vowel_shape_labels)
grey_vowel_position_labels_train, grey_vowel_position_labels_test = split_data(grey_shuffled_vowel_position_labels)
grey_vowel_phoneme_labels_train, grey_vowel_phoneme_labels_test = split_data(grey_shuffled_vowel_phoneme_labels)

In [74]:
#Build the network to recognise hand position
grey_hand_position_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_position_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_position_model.fit(grey_vowel_train, grey_vowel_position_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 32s 961ms/step - loss: 0.6519 - accuracy: 0.7672
Epoch 2/5
33/33 [==============================] - 31s 948ms/step - loss: 0.2453 - accuracy: 0.9243
Epoch 3/5
33/33 [==============================] - 32s 957ms/step - loss: 0.1449 - accuracy: 0.9631
Epoch 4/5
33/33 [==============================] - 30s 910ms/step - loss: 0.1096 - accuracy: 0.9719
Epoch 5/5
33/33 [==============================] - 32s 964ms/step - loss: 0.0863 - accuracy: 0.9835


In [75]:
#Test the network and print it's performance
grey_hand_position_test_loss, grey_hand_position_test_accuracy = grey_hand_position_model.evaluate(grey_vowel_test, grey_vowel_position_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_position_test_loss, grey_hand_position_test_accuracy*100))

4/4 [==============================] - 1s 237ms/step - loss: 0.1382 - accuracy: 0.9826
Test loss: 0.1381930559873581, Test accuracy: 98.26086759567261


In [76]:
#Build network to recognise vowel phoneme
grey_vowel_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_vowel_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_vowel_phoneme_model.fit(grey_vowel_train, grey_vowel_phoneme_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 32s 969ms/step - loss: 1.8379 - accuracy: 0.5063
Epoch 2/5
33/33 [==============================] - 31s 930ms/step - loss: 1.1203 - accuracy: 0.6237
Epoch 3/5
33/33 [==============================] - 30s 915ms/step - loss: 0.9301 - accuracy: 0.6625
Epoch 4/5
33/33 [==============================] - 31s 929ms/step - loss: 0.8245 - accuracy: 0.7129
Epoch 5/5
33/33 [==============================] - 30s 917ms/step - loss: 0.7351 - accuracy: 0.7139


In [77]:
#Test the network and print it's performance
grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy = grey_vowel_phoneme_model.evaluate(grey_vowel_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy*100))

4/4 [==============================] - 1s 254ms/step - loss: 1.0889 - accuracy: 0.5913
Test loss: 1.0889414548873901, Test accuracy: 59.130436182022095


In [78]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_position_decision = grey_hand_position_model(grey_vowel_train)

grey_vowel_phoneme_decision = grey_vowel_phoneme_model(grey_vowel_train)

grey_vowel_decision = join_outputs(grey_hand_position_decision, grey_vowel_phoneme_decision)

In [79]:
#Obtain test vectors to test network on
grey_hand_position_decision_test = grey_hand_position_model(grey_vowel_test)

grey_vowel_phoneme_decision_test = grey_vowel_phoneme_model(grey_vowel_test)

grey_vowel_decision_test = join_outputs(grey_hand_position_decision_test, grey_vowel_phoneme_decision_test)

In [80]:
#Build network to make final decision of phoneme
grey_vowel_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_vowel_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_vowel_decision_model.fit(grey_vowel_decision, grey_vowel_phoneme_labels_train, epochs=30)

Epoch 1/30
33/33 [==============================] - 0s 2ms/step - loss: 3.1442 - accuracy: 0.0873
Epoch 2/30
33/33 [==============================] - 0s 2ms/step - loss: 2.9739 - accuracy: 0.2357
Epoch 3/30
33/33 [==============================] - 0s 1ms/step - loss: 2.7386 - accuracy: 0.4607
Epoch 4/30
33/33 [==============================] - 0s 1ms/step - loss: 2.4316 - accuracy: 0.4597
Epoch 5/30
33/33 [==============================] - 0s 1ms/step - loss: 2.0889 - accuracy: 0.5199
Epoch 6/30
33/33 [==============================] - 0s 1ms/step - loss: 1.7834 - accuracy: 0.6295
Epoch 7/30
33/33 [==============================] - 0s 2ms/step - loss: 1.5529 - accuracy: 0.6440
Epoch 8/30
33/33 [==============================] - 0s 2ms/step - loss: 1.3922 - accuracy: 0.6499
Epoch 9/30
33/33 [==============================] - 0s 1ms/step - loss: 1.2752 - accuracy: 0.6508
Epoch 10/30
33/33 [==============================] - 0s 2ms/step - loss: 1.1879 - accuracy: 0.6518
Epoch 11/30
33/33 [

In [81]:
#Test network and print result
grey_vowel_test_loss, grey_vowel_test_accuracy = grey_vowel_decision_model.evaluate(grey_vowel_decision_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_test_loss, grey_vowel_test_accuracy*100))

4/4 [==============================] - 0s 3ms/step - loss: 0.9820 - accuracy: 0.6000
Test loss: 0.9820446372032166, Test accuracy: 60.00000238418579


In [99]:
#Build SVM classifier for vowel
vowel_svm = svm.SVC(kernel='linear')

vowel_svm.fit(grey_vowel_decision, grey_vowel_phoneme_labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [100]:
#Test SVM classifier and print accuracy
svm_vowel_decision = vowel_svm.predict(grey_vowel_decision_test)

print("Accuracy:",metrics.accuracy_score(grey_vowel_phoneme_labels_test, svm_vowel_decision)*100)

In [57]:
#Build Naive-Bayes classifier for vowel
vowel_nb = naive_bayes.GaussianNB()

vowel_nb.fit(grey_vowel_decision, grey_vowel_phoneme_labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [58]:
#Test Naive-Bayes classifier and print accuracy
nb_vowel_decision = vowel_nb.predict(grey_vowel_decision_test)

print("Accuracy:",metrics.accuracy_score(grey_vowel_phoneme_labels_test, nb_vowel_decision)*100)